In [ ]:
######### Gurobi Code for Production Planning problem



import numpy as np
from gurobipy import*

#production months
i=range(1,13)

#production mode
j=range(1,3)

#selling month
k=range(1,13)

# production capacity
u=[0,800,200]

#demand
d=[0,800,900,1100,400,700,1100,700,900,800,700,800,1100]

#cost
c=[0,10,12]

#define model
M=Model("Production_Planning")

#add decision variable
x=M.addVars(i,j,k,vtype=GRB.INTEGER,name='production_planning')


#set decision variables to zero for i<k and i>k+2
for a in i:
    for b in j:
        for s in k:
            if s<a:
                M.addConstr(x[a,b,s]==0)
            elif s>a+2:
                M.addConstr(x[a,b,s]==0)


#set objective
y=quicksum(c[b] * (quicksum(x[a,b,s] for s in k for a in i)) for b in j)

z=quicksum((s-a)*x[a,b,s] for b in j for a in i for s in k)

obj=y+z
M.setObjective(obj, GRB.MINIMIZE)

# demand of each month should be satisfied
M.addConstrs((quicksum(x[a,b,s] for b in j for a in i) == d[s] for s in k), name="Demand")

# production should not exceed capacity limit
M.addConstrs((quicksum(x[a,b,s] for s in k)<=u[b] for b in j for a in i),name='capacity')

#non-negativity constraint
M.addConstrs((x[a,b,s]>=0 for a in i for b in j for s in k),name="non_negative")

#optimize the model
M.optimize()

#Print objective value

print("The minimum cost that can be incurred is", M.objVal, )

#print decision variable

for a in i:
    for b in j:
        for s in k:
            print("the units produced in month", a, "via production mode", b ,"and sold in month", s, "is" ,x[a,b,s].X)

